## Eksperimen 1: Visual Problem Generation (VLLM)

Generate soal visual dengan VLLM.

In [1]:
print("Setup VLLM experiment...")

Setup VLLM experiment...


In [2]:
import time
import json
import re
import datetime
from vllm import LLM, SamplingParams

ModuleNotFoundError: No module named 'vllm'

In [ ]:
MODEL_NAME = "facebook/opt-125m" 

Using device: cpu


In [ ]:
def extract_last_json(text: str):
    matches = re.findall(r"\{[\s\S]*\}", text)
    if not matches:
        return None

    for cand in reversed(matches):
        try:
            return json.loads(cand)
        except json.JSONDecodeError:
            continue
    return None


In [ ]:
def run_experiment_vllm(plan, date_str):
    prompt = f"""
You are an AI teacher and problem generator.

Plan: {plan}
Date: {date_str}

Task:
Generate ONE concrete math problem based on the plan.
If an image helps understanding, describe the image clearly in the problem text.

Rules:
- You MUST fill all fields with real content.
- Do NOT repeat placeholders or templates.
- Output ONLY valid JSON.
- No explanation outside JSON.

Required JSON format:
{{
  "soal": "...",
  "image_description": "...",
  "solution": "..."
}}
"""
    
    sampling_params = SamplingParams(
        temperature=0.2,
        top_p=0.9,
        max_tokens=1024
    )

    print(f"Loading model {MODEL_NAME} using vLLM...")
    llm = LLM(model=MODEL_NAME, trust_remote_code=True)

    print("Starting generation...")
    start_time = time.time()

    outputs = llm.generate([prompt], sampling_params)

    end_time = time.time()
    inference_time = end_time - start_time

    result_text = outputs[0].outputs[0].text

    print(f"Inference finished in {inference_time:.4f} seconds.")
    return result_text, inference_time


In [ ]:
plan_input = "Geometri: Menghitung luas bangun datar gabungan (Persegi dan Segitiga)"
date_input = datetime.date.today().strftime("%Y-%m-%d")

try:
    result_text, duration = run_experiment_vllm(plan_input, date_input)

    print("\n--- Raw Model Output ---")
    print(result_text)

    print("\n--- Parsed JSON ---")
    parsed = extract_last_json(result_text)
    if parsed:
        print(json.dumps(parsed, indent=4, ensure_ascii=False))
    else:
        print("JSON parsing failed, raw text returned.")

except Exception as e:
    print(f"Error running vLLM: {e}")
    print("Pastikan vLLM dan CUDA tersedia.")


Starting inference (Ollama)...
Inference finished in 9.6474 seconds.

--- Raw Model Output ---
{
  "soal": "Diberikan persegi dengan sisi panjang 8 cm dan segitiga yang berada di dalamnya, dengan alas 6 cm dan tinggi 8 cm. Hitunglah luas bangun datar gabungan ini!",
  "image_description": "Gambar sebuah persegi dengan sisi panjang 8 cm, dan di dalamnya terdapat segitiga dengan alas 6 cm dan tinggi 8 cm.",
  "solution": "Luas persegi = sisi^2 = 8^2 = 64 cm^2. Luas segitiga = (alas * tinggi) / 2 = (6 * 8) / 2 = 24 cm^2. Jadi, luas bangun datar gabungan ini adalah: 64 + 24 = 88 cm^2."
}

--- Parsed JSON ---
{
    "soal": "Diberikan persegi dengan sisi panjang 8 cm dan segitiga yang berada di dalamnya, dengan alas 6 cm dan tinggi 8 cm. Hitunglah luas bangun datar gabungan ini!",
    "image_description": "Gambar sebuah persegi dengan sisi panjang 8 cm, dan di dalamnya terdapat segitiga dengan alas 6 cm dan tinggi 8 cm.",
    "solution": "Luas persegi = sisi^2 = 8^2 = 64 cm^2. Luas segitiga 